In [ ]:
!pip install streamlit
!pip install pymongo
!pip install textblob
!pip install torch
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 22.5 MB/s eta 0:00:00


In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from pymongo import MongoClient
from textblob import TextBlob
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
from datasets import load_dataset

dataset = load_dataset("setfit/sst5")
print(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/421 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl: 0.00B [00:00, ?B/s]

dev.jsonl: 0.00B [00:00, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 8544
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 1101
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2210
    })
})


In [ ]:
print(dataset['train'][0])
print(dataset['train']['text'][:5])
print(dataset['train']['label'][:5])

{'text': 'a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films', 'label': 4, 'label_text': 'very positive'}
['a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films', 'apparently reassembled from the cutting-room floor of any given daytime soap .', "they presume their audience wo n't sit still for a sociology lesson , however entertainingly presented , so they trot out the conventional science-fiction elements of bug-eyed monsters and futuristic women in skimpy clothes .", 'the entire movie is filled with deja vu moments .', 'this is a visually stunning rumination on love , memory , history and the war between art and commerce .']
[4, 1, 1, 2, 3]


In [ ]:
from transformers import AutoTokenizer

model_name = "bert-base-uncased" 
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/8544 [00:00<?, ? examples/s]

Map:   0%|          | 0/1101 [00:00<?, ? examples/s]

Map:   0%|          | 0/2210 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 5  
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.1 MB/s eta 0:00:00


In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from evaluate import load

accuracy_metric = load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)


from transformers import TrainingArguments, IntervalStrategy

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    logging_steps=50,
    eval_strategy=IntervalStrategy.STEPS,  
    eval_steps=200,              
    save_steps=200,
    save_total_limit=3,
    metric_for_best_model="accuracy",
    load_best_model_at_end=True   #
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]

)

/tmp/ipython-input-899306005.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()
trainer.save_model("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sanketsdeore (sanketsdeore-international-institute-of-information-tech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy
200,1.219900,1.213145,0.457766
400,1.186100,1.147289,0.491371
600,0.969300,1.115967,0.502271
800,0.929100,1.135538,0.519528
1000,0.940200,1.133449,0.504995
1200,0.712400,1.264792,0.497729


('./sentiment_model/tokenizer_config.json',
 './sentiment_model/special_tokens_map.json',
 './sentiment_model/vocab.txt',
 './sentiment_model/added_tokens.json',
 './sentiment_model/tokenizer.json')

In [ ]:
!pip install streamlit-autorefresh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.8/700.8 kB 42.3 MB/s eta 0:00:00


In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from pymongo import MongoClient
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer

from streamlit.runtime.scriptrunner import add_script_run_ctx
from streamlit_autorefresh import st_autorefresh

    st_autorefresh(interval=60000, limit=None, key="auto_refresh")

    st.set_page_config(page_title="eConsultation Dashboard", page_icon="📊", layout="centered")

    st.markdown("""
        <style>
            .block-container {
                max-width: 900px;
                padding: 2rem 1rem 0.5rem 1rem;
            }
                div[data-testid="stMetric"] {
                    background: #ffffffcc;
                    border: 1px solid #ddd;
                    border-radius: 10px;
                    padding: 6px;
                    margin: 2px;
                    box-shadow: 0px 1px 2px rgba(0,0,0,0.05);
                }
                [data-testid="stMetricLabel"] { font-size: 12px; color: #555; }
                [data-testid="stMetricValue"] { font-size: 16px; font-weight: 600; color: #222; }
        </style>
    """, unsafe_allow_html=True)

    sentiment_colors = {
        "Very Positive": "#2e7d32",
        "Positive": "#66bb6a",
        "Neutral": "#fbc02d",
        "Negative": "#ef5350",
        "Very Negative": "#b71c1c"
    }

model_path = "./sentiment_model"
use_hf_model = False

    try:
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModelForSequenceClassification.from_pretrained(model_path)
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model.to(device)
        use_hf_model = True
    except:
        use_hf_model = False

client = MongoClient("")
db = client["econsultation"]
collection = db["comments"]

    all_comments_cursor = collection.find({}, {"commentText": 1, "_id": 0, "stakeholderType": 1, "submissionDate":1})
    data = []
    for c in all_comments_cursor:
        data.append({
            "comment": c.get("commentText",""),
            "stakeholderType": c.get("stakeholderType","Unknown"),
            "submissionDate": c.get("submissionDate")
        })

df_raw = pd.DataFrame(data)

st.title("📊 eConsultation Dashboard")

first_doc = collection.find_one({}, {"_id": 0, "documentTitle": 1})
document_text = first_doc["documentTitle"] if first_doc and "documentTitle" in first_doc else "Untitled Document"
st.markdown(f"<h4>Document Title: {document_text}</h4>", unsafe_allow_html=True)

stakeholder_options = ["All"] + sorted(df_raw['stakeholderType'].unique())
selected_stakeholder = st.selectbox("Filter by Stakeholder Type:", stakeholder_options)

sentiment_options = ["All","Positive","Negative"]
selected_sentiment = st.selectbox("Filter by Sentiment:", sentiment_options)

keyword = st.text_input("Search keyword:")

df = df_raw.copy()
    if selected_stakeholder != "All":
        df = df[df['stakeholderType'] == selected_stakeholder]
    if keyword.strip():
        df = df[df['comment'].str.contains(keyword, case=False)]

results = []
label_map = {0: "Very Negative", 1: "Negative", 2: "Neutral", 3: "Positive", 4: "Very Positive"}

for comment in df["comment"]:
    if use_hf_model:
        inputs = tokenizer(comment, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
            pred = torch.argmax(probs, dim=-1).item()
        sentiment = label_map[pred]
        confidence = probs[0][pred].item()
    else:
        polarity = TextBlob(comment).sentiment.polarity
        if polarity > 0.5: sentiment = "Very Positive"
        elif polarity > 0: sentiment = "Positive"
        elif polarity == 0: sentiment = "Neutral"
        elif polarity > -0.5: sentiment = "Negative"
        else: sentiment = "Very Negative"
        confidence = round(abs(polarity),3)
    results.append({"comment": comment, "sentiment": sentiment, "confidence": round(confidence,3)})

df['sentiment'] = [r['sentiment'] for r in results]

if selected_sentiment != "All":
    if selected_sentiment == "Positive":
        df = df[df['sentiment'].isin(["Positive", "Very Positive"])]
    elif selected_sentiment == "Negative":
        df = df[df['sentiment'].isin(["Negative", "Very Negative"])]

st.markdown("---")
st.subheader("Key Performance Indicators")
col1, col2, col3, col4 = st.columns(4)
total_comments = len(df)
pos_count = len(df[df["sentiment"].isin(["Positive", "Very Positive"])])
neg_count = len(df[df["sentiment"].isin(["Negative", "Very Negative"])])
neu_count = len(df[df["sentiment"] == "Neutral"])

col1.metric("Total", total_comments)
col2.metric("Positive", pos_count)
col3.metric("Neutral", neu_count)
col4.metric("Negative", neg_count)

st.markdown("---")
st.subheader("📋 Comments with Sentiment")

if not df.empty:
    display_df = df[["comment", "stakeholderType", "sentiment"]].copy()
    display_df.rename(columns={
        "comment": "Comment",
        "stakeholderType": "Stakeholder",
        "sentiment": "Sentiment"
    }, inplace=True)

    st.dataframe(display_df, use_container_width=True, hide_index=True)
else:
    st.info("No comments available for display with current filters.")

col5, col6 = st.columns(2)

with col5:
    st.write("Sentiment Distribution")
    sentiment_counts = df["sentiment"].value_counts()
    if not sentiment_counts.empty:
        fig, ax = plt.subplots(figsize=(4,3))
        sentiment_counts.plot(
            kind="bar",
            color=[sentiment_colors.get(s, sentiment_colors['Neutral']) for s in sentiment_counts.index],
            ax=ax
        )
        ax.set_ylabel("")
        ax.set_xlabel("")
        ax.tick_params(axis="x", labelrotation=25, colors="white", labelsize=8)
        ax.tick_params(axis="y", colors="white", labelsize=8)
        for spine in ax.spines.values():
            spine.set_visible(False)
        fig.patch.set_alpha(0)
        ax.set_facecolor("none")
        st.pyplot(fig, transparent=True)
    else:
        st.info("⚠️ No comments available for bar chart with current filters.")

with col6:
    st.write("Sentiment Breakdown")
    sentiment_counts = df["sentiment"].value_counts()
    if not sentiment_counts.empty:
        fig2, ax2 = plt.subplots(figsize=(4,3))
        sentiment_counts.plot(
            kind="pie",
            autopct="%1.0f%%",
            textprops={'fontsize': 8,'color':'white'},
            colors=[sentiment_colors.get(s, sentiment_colors['Neutral']) for s in sentiment_counts.index],
            ax=ax2
        )
        ax2.set_ylabel("")
        ax2.set_title("")
        for spine in ax2.spines.values():
            spine.set_visible(False)
        fig2.patch.set_alpha(0)
        ax2.set_facecolor("none")
        st.pyplot(fig2, transparent=True)
    else:
        st.info("⚠️ No comments available for pie chart with current filters.")

import streamlit as st
from transformers import pipeline, T5Tokenizer
import google.generativeai as genai

@st.cache_resource
def load_models():
    """Load T5 and Gemini models"""
    t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
    t5_summarizer = pipeline("summarization", model="t5-small", tokenizer=t5_tokenizer)

    try:
        gemini_api_key = ""
        genai.configure(api_key=gemini_api_key)
        gemini_model = genai.GenerativeModel('gemini-1.5-flash')
    except Exception as e:
        st.error(f"⚠️ Could not configure Gemini. Error: {e}")
        gemini_model = None


    return t5_tokenizer, t5_summarizer, gemini_model

t5_tokenizer, t5_summarizer, gemini_model = load_models()

def generate_summary(comment_text: str) -> str:
    """
    Summarizes comments intelligently:
    - Short comments (<40 words): Gemini LLM (grammar fix + summary)
    - Long comments: T5 summarizer
    """
    word_count = len(comment_text.split())

    if word_count < 40:
        if gemini_model is None:
            return "Gemini not available. Cannot summarize short text."

        prompt = f"""
        Correct spelling/grammar and summarize into one concise sentence.

        Original Comment: "{comment_text}"
        Cleaned Summary:
        """
        try:
            response = gemini_model.generate_content(prompt)
            return response.text.strip()
        except Exception as e:
            return f"Gemini summarization failed: {e}"

    else:
        tokens = t5_tokenizer.encode(comment_text, return_tensors="pt", truncation=False)[0]

        if len(tokens) <= 512:
            summary = t5_summarizer(comment_text, max_length=150, min_length=30, do_sample=False)
            return summary[0]['summary_text']
        else:
            return "⚠️ Comment too long. Add chunking logic for very large texts."

st.markdown("---")
st.subheader("✨ AI Comment Summarizer")

if not df.empty:
    selected_comment = st.selectbox("Select a comment to summarize:", df["comment"].tolist())

    if st.button("Generate Summary"):
        with st.spinner("AI is summarizing..."):
            summary = generate_summary(selected_comment)
            st.success("Generated Summary:")
            st.write(summary)
else:
    st.info("No comments available for summarization with current filters.")

st.markdown("---")
st.subheader("Top Keywords in Comments")
all_text = " ".join(df['comment'])
vectorizer = CountVectorizer(stop_words='english', max_features=15)
X = vectorizer.fit_transform([all_text])
keywords = vectorizer.get_feature_names_out()
counts = X.toarray().sum(axis=0)
top_keywords_df = pd.DataFrame({'keyword': keywords,'count':counts}).sort_values('count',ascending=True)

fig3, ax3 = plt.subplots(figsize=(6,3))
ax3.barh(top_keywords_df['keyword'], top_keywords_df['count'], color="#4a90e2")
ax3.set_xlabel("Frequency", fontsize=8, color='white')
ax3.tick_params(axis="x", labelsize=6, colors='white')
ax3.tick_params(axis="y", labelsize=8, colors='white')
for spine in ax3.spines.values(): spine.set_visible(False)
fig3.patch.set_alpha(0)
ax3.set_facecolor("none")
st.pyplot(fig3, transparent=True)

st.markdown("---")
st.subheader("Word Cloud")

all_text_wc = " ".join(df['comment'].astype(str))

if all_text_wc.strip():
    wc = WordCloud(
        width=1600, height=600,   
        background_color=None,
        mode="RGBA",
        colormap="viridis",       
        max_words=100
    ).generate(all_text_wc)

    fig_wc, ax_wc = plt.subplots(figsize=(12, 5)) 
    ax_wc.imshow(wc, interpolation="bilinear")
    ax_wc.axis("off")
    fig_wc.tight_layout(pad=0.5)
    st.pyplot(fig_wc, transparent=True, use_container_width=True)
else:
    st.info("No comments available for word cloud.")


# # Negative Word Cloud
# with col_neg:
#     text_neg = " ".join(df[df["sentiment"].isin(["Negative", "Very Negative"])]["comment"].astype(str))
#     if text_neg.strip():
#         st.markdown("**Negative Comments**")
#         wc_neg = WordCloud(width=600, height=500, background_color=None, mode="RGBA",
#                            colormap="Reds", max_words=40).generate(text_neg)
#         fig_neg, ax_neg = plt.subplots(figsize=(3.5, 2))
#         ax_neg.imshow(wc_neg, interpolation="bilinear")
#         ax_neg.axis("off")
#         fig_neg.tight_layout(pad=0.2)
#         st.pyplot(fig_neg, transparent=True)
#     else:
#         st.info("No negative comments for selected filters.")



Overwriting app.py


The Streamlit app is restarting. Please use the new public URL above to access the updated dashboard.

Let me know if you have any other questions or if there's anything else I can help you with!

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import conf

conf.get_default().auth_token = "32se5kQrH2acY9ZibgtvlpgT5Z4_6FFD36zhq1NtdM7kfNsBp"


In [ ]:
from pyngrok import ngrok

ngrok.kill()
public_url = ngrok.connect(8501)
print(f"🔗 Public URL: {public_url}")

🔗 Public URL: NgrokTunnel: "https://02f05aa7a49a.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!streamlit run app.py &>/dev/null &